In [ ]:
!pip install pyspark==3.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 17.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880745 sha256=f66efe71dfaa5658555891fc58323f59c519df7f061fffa1112b5a343338f94c
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import sys
import re
import numpy as np

from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName("WikiDictionary")
sc = SparkContext(conf=conf)

# Set the file paths on your local machine
#wikiPagesFile = "/WikipediaPagesOneDocPerLine1000LinesSmall.txt"
#wikiCategoryFile = "/wiki-categorylinks-small (1).csv.bz2"
wikiPagesFile = sys.argv[1]
wikiCategoryFile = sys.argv[2]

# Read two files into RDDs
wikiCategoryLinks = sc.textFile(wikiCategoryFile)
wikiCats = wikiCategoryLinks.map(lambda x: x.split(",")).map(lambda x: (x[0].replace('"', ''), x[1].replace('"', '')))

wikiPages = sc.textFile(wikiPagesFile)

# Assumption: Each document is stored in one line of the text file
# We need this count later ...
numberOfDocs = wikiPages.count()

print(numberOfDocs)

1000


In [ ]:
validLines = wikiPages.filter(lambda x: 'id' in x and 'url=' in x)

keyAndText = validLines.map(lambda x: (x[x.index('id="') + 4: x.index('" url=')], x[x.index('">') + 2:][:-6]))

def buildArray(listOfIndices):
    returnVal = np.zeros(20000)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    mysum = np.sum(returnVal)
    returnVal = np.divide(returnVal, mysum)
    return returnVal

keyAndListOfWords = keyAndText.map(lambda x: (str(x[0]), re.compile('[^a-zA-Z]').sub(' ', x[1]).lower().split()))

allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))

allCounts = allWords.reduceByKey(lambda a, b: a + b)

topWords = allCounts.top(20000, key=lambda x: x[1])
#task 1.1
print("Top 3 in top 20,000 words:", topWords[:3])
topWordsK = sc.parallelize(range(20000))

dictionary = topWordsK.map(lambda x: (topWords[x][0], x))
dictionary_broadcast = sc.broadcast(dictionary.collectAsMap())

def build_zero_one_array(listOfWords):
    returnVal = np.zeros(20000)
    for word in listOfWords:
        index = dictionary_broadcast.value.get(word)
        if index is not None:
            returnVal[index] = 1
    return returnVal




def stringVector(x):
    returnVal = str(x[0])
    for j in x[1]:
        returnVal += ',' + str(j)
    return returnVal

def cousinSim(x, y):
    normA = np.linalg.norm(x)
    normB = np.linalg.norm(y)
    return np.dot(x, y) / (normA * normB)

Top 3 in top 20,000 words: [('the', 74530), ('of', 34512), ('and', 28479)]


In [ ]:




allWordsWithDocID = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

allDictionaryWords = allWordsWithDocID.join(dictionary)

justDocAndPos = allDictionaryWords.map(lambda x: (x[1][0], x[1][1]))

allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))
#task1.2
print("First 3 elements in RDD with docID and dictionary positions:", allDocsAsNumpyArrays.take(3))

#task 2
zeroOrOne = allDocsAsNumpyArrays.map(lambda x: (x[0], build_zero_one_array(x[1])))
print("First 3 elements in Term Frequency (TF) array:", zeroOrOne.take(3))

dfArray = zeroOrOne.reduce(lambda x1, x2: ("", np.add(x1[1], x2[1])))[1]

multiplier = np.full(20000, numberOfDocs)

idfArray = np.log(np.divide(multiplier, dfArray + 1))
print("Inverse Document Frequency (IDF) array:", idfArray)

allDocsAsNumpyArraysTFidf = allDocsAsNumpyArrays.map(lambda x: (x[0], np.multiply(x[1], idfArray)))
print("First 3 elements in TF-IDF matrix of the corpus:", allDocsAsNumpyArraysTFidf.take(3))

featuresRDD = wikiCats.join(allDocsAsNumpyArraysTFidf).map(lambda x: (x[1][0], x[1][1]))

featuresRDD.cache()

def getPrediction(textInput, k):
    myDoc = sc.parallelize(('', textInput))
    wordsInThatDoc = myDoc.flatMap(lambda x: ((j, 1) for j in re.compile('[^a-zA-Z]').sub(' ', x).lower().split()))
    allDictionaryWordsInThatDoc = dictionary.join(wordsInThatDoc).map(lambda x: (x[1][1], x[1][0])).groupByKey()
    myArray = buildArray(allDictionaryWordsInThatDoc.top(1)[0][1])
    myArray = np.multiply(myArray, idfArray)
    distances = featuresRDD.map(lambda x: (x[0], np.dot(x[1], myArray)))
    topK = distances.top(k, lambda x: x[1])
    docIDRepresented = sc.parallelize(topK).map(lambda x: (x[0], 1))
    numTimes = docIDRepresented.reduceByKey(lambda a, b: a + b)
    return numTimes.top(k, lambda x: x[1])

#print(getPrediction('Sport Basketball Volleyball Soccer', 10))

First 3 elements in RDD with docID and dictionary positions: [('431952', array([0.08850932, 0.02950311, 0.04658385, ..., 0.        , 0.        ,
       0.        ])), ('431971', array([0.08553655, 0.02488336, 0.05132193, ..., 0.        , 0.        ,
       0.        ])), ('431989', array([0.09074244, 0.03574702, 0.03849679, ..., 0.        , 0.        ,
       0.        ]))]
First 3 elements in Term Frequency (TF) array: [('431952', array([0., 0., 0., ..., 0., 0., 0.])), ('431971', array([0., 0., 0., ..., 0., 0., 0.])), ('431989', array([0., 0., 0., ..., 0., 0., 0.]))]
Inverse Document Frequency (IDF) array: [6.90775528 6.90775528 6.90775528 ... 6.90775528 6.90775528 6.90775528]
First 3 elements in TF-IDF matrix of the corpus: [('431952', array([0.6114007 , 0.20380023, 0.32178984, ..., 0.        , 0.        ,
       0.        ])), ('431971', array([0.59086554, 0.17188816, 0.35451932, ..., 0.        , 0.        ,
       0.        ])), ('431989', array([0.62682656, 0.24693167, 0.26592642,

In [ ]:
print(getPrediction('What is the capital city of Australia?', 10))

[('All_set_index_articles', 2), ('Disambiguation_pages_with_short_description', 1), ('Russian_Soviet_Federative_Socialist_Republic', 1), ('All_article_disambiguation_pages', 1), ('All_disambiguation_pages', 1), ('Disambiguation_pages', 1), ('Articles_with_short_description', 1), ('Royal_Navy_ship_names', 1), ('Set_indices_on_ships', 1)]


In [ ]:
#print(getPrediction('How many goals Vancouver score last year?', 10))

[('19th_century_in_science', 2), ('1840s_in_science', 2), ('Articles_with_short_description', 2), ('1841_in_science', 1), ('1830s_in_science', 1), ('1839_in_science', 1), ('1840_in_science', 1)]
